In [2]:
import time
import requests
import numpy as np
import pandas as pd
from tqdm import tqdm
import chromedriver_binary
from bs4 import BeautifulSoup
from selenium import webdriver


df = pd.read_csv('dataset_raw.csv')
df.head()

driver = webdriver.Chrome()

data = []

### 1. Scraping Data

In [3]:
driver.get('https://www.kaggle.com/ashishjangra27/')

time.sleep(5)

for link in tqdm(df['dataset_link']):       # Going through each link
    
    driver.get(link)                        # Sending Request to each link
    
    time.sleep(1)
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    title = soup.find('h1').text.strip()
    
    try:
        licence        =   soup.find('a',target = '_blank').text
    except:
        licence = np.nan
    try:
        views          =   int(soup.find_all('div', style = 'margin-bottom: 36px;')[0].text[5:].strip())
    except:
        views = np.nan
    try:
        downloads      =   int(soup.find_all('div', style = 'margin-bottom: 36px;')[1].text[9:].strip())
    except:
        downloads = np.nan
    try:
        contributions  =   int(soup.find_all('div', style = 'margin-bottom: 36px;')[3].text[25:].strip())
    except:
        contributions = np.nan
    try:
        notebooks      =   int(soup.find_all('div', style = 'margin-bottom: 36px;')[4].text[9:].strip())
    except:
        notebooks = np.nan
    try:
        nb_comments    =   int(soup.find_all('div', style = 'margin-bottom: 36px;')[5].text[17:].strip())
    except:
        nb_comments = np.nan
        
    data.append([title, views, downloads, contributions, notebooks, nb_comments, licence])

100%|███████████████████████████████████████████| 50/50 [02:36<00:00,  3.13s/it]


### 2. Creating DataFrame

In [10]:
df = pd.DataFrame(data, columns = ['title','views','downloads','contributions',
                                   'notebooks','nb_comments','licence'])

df.head()

,title,views,downloads,contributions,notebooks,nb_comments,licence
0,Netflix TV Shows and Movies,32038.0,5977.0,14.0,11.0,5.0,CC0: Public Domain
1,Supermarket store branches sales analysis,27866.0,5357.0,8.0,8.0,57.0,CC BY-SA 3.0
2,Student Performance Dataset,11944.0,2336.0,4.0,1.0,17.0,CC0: Public Domain
3,Top Hits Spotify from 2000-2019,26987.0,5026.0,16.0,14.0,18.0,
4,Amex Competition Data in Parquet Format,3875.0,612.0,8.0,9.0,14.0,CC0: Public Domain


### 3. Removing Nan Values

In [15]:
df = df.dropna()

### 4. Dealing with Un-Licenced Dataset

In [22]:
licence = []

for i in df['licence']:
    
    if(i == ''):
        licence.append('Unlicenced')
    else:
        licence.append(i)
        
df['licence'] = licence

df.head()

/var/folders/rq/lwddjvtn5n9gjw05hwrg3hp80000gn/T/ipykernel_12122/2157947686.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['licence'] = licence


,title,views,downloads,contributions,notebooks,nb_comments,licence
0,Netflix TV Shows and Movies,32038.0,5977.0,14.0,11.0,5.0,CC0: Public Domain
1,Supermarket store branches sales analysis,27866.0,5357.0,8.0,8.0,57.0,CC BY-SA 3.0
2,Student Performance Dataset,11944.0,2336.0,4.0,1.0,17.0,CC0: Public Domain
3,Top Hits Spotify from 2000-2019,26987.0,5026.0,16.0,14.0,18.0,Unlicenced
4,Amex Competition Data in Parquet Format,3875.0,612.0,8.0,9.0,14.0,CC0: Public Domain


In [24]:
df.to_csv('dataset.csv',index = False)